# Honours Project Notebook.

This notebook contains the code for my honours project. 

To run the application, `ollama` must be installed on the system. The `ollama` daemon must be run in the background using `ollama serve`. 

In [10]:
# Dependencies:
import ollama
import pandas as pd
from pprint import pprint

In [ ]:
MODELSUSED = ["mistral:latest", "falcon3:latest", "qwen2.5:latest"]

modelNames = [model.get("model") for model in ollama.list().models]

# Get list of models names on the system.

for i in MODELSUSED:
    if i not in modelNames:
        raise Exception(f"Model {i} not on system.")


SYSTEM = """You help correct radiology report errors. These include transcription errors, internal inconsistencies, insertion statements and translation errors. For each mistake, show the incorrect words and explain what the problem is."""


EXAMPLE = """
Report: 
Clinical Information Provided:
50-year-old male. fall from 7feet high when ladder gave way. Landed on back. c/o lpower rib and back pain. o/e tender BL lower limbs and flank, guarding. pneumothorax. intra-abdominal injury
.
Technique:
CT chest/abdomen/pelvis with dual arterial portal venous phase enhancement.
.
Findings:
Chest:
pneumothorax, haemothorax, or pulmonary contusion.
Old left fourth rib fracture.  No acute bony injury to sternum or thoracic spine.  Wide patent bony spinal canal.
.
Abdomen/pelvis:
No intra-abdominal/pelvic free gas or fluid.
No significant abnormality seen in solid upper abdominal organs.  Normal appearance of partially filled urinary bladder.
Colonic diverticulosis with no active diverticulitis.  Remainder of unprepared large and small bowel loops are within normal limits.
No acute bony injury.  Degenerative changes in thoracolumbar spine and sacroiliac joints. Wide patent bony spinal canal.
.
Vascular:
Normal appearance of thoracic aorta with conventional configuration of aortic arch.
Normal appearance of abdominal aorta and its major branches.  Normal opacification of iliac vessels and visualised proximal femoral vessels.
.
Conclusion:
No pneumothorax.  No intra-abdominal traumatic injury.

Correction: Omission as no pneumothorax stated in the conclusion but findings say pneumothorax.
The radiology report will be below.
"""
dataframe = pd.read_csv("datasets/testing_data.csv")

removedCorrection = dataframe["Removed Correction"]

correctedData = dataframe["Re-dictated"]

errors = dataframe["Type of Error"]
# Sanity check.
for i in range(len(removedCorrection)):
    if removedCorrection[i] == correctedData[i]:
        raise ValueError(
            f"Correction not removed properly in {i} : \n{removedCorrection[i]}"
        )

for i in range(len(errors)):
    errorList = errors[i].split(";")
    errorNumber = len(errorList)
    # print(f"{i} : {errorNumber} errors.")
# display(errors)

In [ ]:
# Feed each report into the models.
temp = removedCorrection.head(1)
reportDict: dict[str, list[str]] = {
    "Original report": temp,
    "mistral:latest": [],
    "falcon3:latest": [],
    "qwen2.5:latest": [],
}

for report in temp:
    for name in MODELSUSED:
        generated = ollama.generate(
            name, prompt=SYSTEM + report, options={"temperature": 0}
        )
        reportDict[name].append(generated["response"])

# pprint(reportDict)

In [ ]:
# Convert dictionary into CSV file.

tempData = pd.DataFrame().from_dict(reportDict)

tempData.to_csv("datasets/preliminary_eval_results.csv")

display(tempData)

,Original report,mistral:latest,falcon3:latest,qwen2.5:latest
0,Clinical Information Provided:\n50-year-old fe...,"1. Incorrect Words: ""active haemorrhage at tim...",**Radiology Report Analysis and Corrections**\...,Certainly! Let's go through the report and ide...
